# Bibliotecas

In [1]:
from metapy_toolbox import *
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)

from fundacao import *

# Carregando dados

In [2]:
df = pd.read_excel(r"/home/wmpjrufg/Documents/fundaIA/assets/toy_problem.xlsx") # Prof. Wanderlei
n_comb = 3
f_ck = 25000
cob_m = 0.04
df

,Elemento,ap (m),bp (m),spt,solo,xg (m),yg (m),Fz-c1,Mx-c1,My-c1,Fz-c2,Mx-c2,My-c2,Fz-c3,Mx-c3,My-c3
0,P16,0.25,1.2,43,argila,27.75,18.065,377.3,-1.9,4.4,259.2,-27.8,0.1,383.8,27.7,0.2
1,P04,0.30,1.5,35,argila,24.85,25.105,855.5,-3.7,9.2,891.9,-60.0,0.6,908.5,-36.9,0.4
2,P05,0.25,1.2,45,argila,24.85,25.255,478.6,-3.1,5.0,496.0,-27.6,0.7,508.0,-27.6,0.7


# Pop inicial de possíveis soluções

In [3]:
n_population = 20000
seed = 42

In [4]:
x_lower = [0.60] * len(df) * 3
x_upper = [10.0] * len(df) * 3
n_dimensions = len(df) * 3
pop2d = initial_population_01(n_population, n_dimensions, x_lower, x_upper, seed, use_lhs=True)

# Gerando dataset completo

In [9]:
cols = []
for i in range(len(df)):
    cols.append(f'hx{i}')
    cols.append(f'hy{i}')
    cols.append(f'hz{i}')
df_pop = pd.DataFrame(pop2d, columns=cols)
args = [df, n_comb, f_ck, cob_m]
def wrapper(x):
    return obj_teste(x, args)[0]
vol = Parallel(n_jobs=-1)(delayed(wrapper)(x) for x in pop2d)
df_pop['volume (m3)'] = vol
df_pop

,hx0,hy0,hz0,hx1,hy1,hz1,hx2,hy2,hz2,volume (m3)
0,3.977056,5.536204,1.549936,4.779852,7.568646,2.342771,5.790322,1.435291,7.141400,1.015304e+06
1,2.463338,7.963786,3.510744,3.649057,9.076063,1.348032,8.819253,3.044209,5.894990,7.682060e+05
2,9.573791,6.173433,4.290554,8.484553,7.910394,1.187550,0.773534,6.896029,0.859691,1.080190e+06
3,9.331639,2.656647,3.729409,8.981630,3.328365,6.033047,9.346526,2.830869,8.983771,1.811998e+06
4,9.865989,5.388136,4.414413,1.199875,5.038475,6.942259,8.702471,9.538313,7.066809,1.127523e+06
...,...,...,...,...,...,...,...,...,...,...
19995,4.141515,7.217091,3.406137,3.432898,1.181040,5.979050,6.652023,7.587652,8.792934,1.455116e+06
19996,9.813267,2.127059,0.735609,9.639918,3.356010,3.564895,6.238583,7.295801,8.671150,1.107679e+06
19997,8.225736,6.436045,4.085483,5.023974,0.976906,2.925458,4.058613,3.518537,2.578905,1.567825e+06
19998,4.700392,9.470813,5.761113,3.333187,7.883072,5.653579,1.500235,1.809298,2.647043,1.214356e+06


In [ ]:
# Save the data splits
fold_data = {'metadata': f"This pkl contains the data splits",
    'x_train': x_train,
    'x_test': x_test,
    'y_train': y_train,
    'y_test': y_test
}
data_filename = f'models/fold_data_{model_name}_fold_{i+1}.pkl'
joblib.dump(fold_data, data_filename)

NameError: name 'x_train' is not defined

In [ ]:
import joblib
name_best_model = 'models/model_HistGradientBoosting_fold_4.pkl'
model = joblib.load(name_best_model)

# Aprendizado de máquina do dataset

In [14]:
x = df_pop.drop(columns=['volume (m3)'])
y = df_pop[['volume (m3)']].reset_index(drop=True)

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.2, random_state=seed)

In [ ]:
os.makedirs("models", exist_ok=True)

In [ ]:
model_name= "DF"
fold_id= 1
fold_data= {
    "metadata" : "Data split holdout 80/20; random_state=42",
    "x_train": x_train,
    "x_test": x_test,
    "y_train": y_train,
    "y_test": y_test
}
data_filename = f'models/fold_data_{model_name}_fold_{fold_id}.pkl'
joblib.dump(fold_data, data_filename)
print(f"Data split saved to {data_filename}")

Data split saved to models/fold_data_DF_fold_1.pkl
